## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root ./artifacts`

In [1]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts/0', creation_time=1716630209592, experiment_id='0', last_update_time=1716630209592, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("scenario-2")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/05/25 09:45:11 INFO mlflow.tracking.fluent: Experiment with name 'scenario-2' does not exist. Creating a new experiment.


default artifacts URI: '/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts/1/6e2491bbc96340b7b0061be7354fadea/artifacts'


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts/1', creation_time=1716630311469, experiment_id='1', last_update_time=1716630311469, lifecycle_stage='active', name='scenario-2', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts/0', creation_time=1716630209592, experiment_id='0', last_update_time=1716630209592, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [6]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [54]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1716630728413, description='', last_updated_timestamp=1716632575102, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1716632575102, current_stage='None', description='', last_updated_timestamp=1716632575102, name='iris-classifier', run_id='6e2491bbc96340b7b0061be7354fadea', run_link='', source='/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts/1/6e2491bbc96340b7b0061be7354fadea/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>], name='iris-classifier', tags={}>]


In [60]:
import pprint
import pandas as pd

pprint.pp(client.search_runs(experiment_ids='1'))

[<Run: data=<RunData: metrics={'accuracy': 0.96}, params={'C': '0.1', 'random_state': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "6e2491bbc96340b7b0061be7354fadea", '
                             '"artifact_path": "models", "utc_time_created": '
                             '"2024-05-25 09:45:11.659734", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.11.9", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.4.2", '
                             '"serialization_format": "cloudpickle", "code": '
                             'null}}, "model_uuid": '
                             '"26a117fcb3

In [64]:
pprint.pp(client.search_runs(experiment_ids='1')[0].to_dictionary(), indent=4)

{   'info': {   'artifact_uri': '/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts/1/6e2491bbc96340b7b0061be7354fadea/artifacts',
                'end_time': 1716630314138,
                'experiment_id': '1',
                'lifecycle_stage': 'active',
                'run_id': '6e2491bbc96340b7b0061be7354fadea',
                'run_name': 'able-shoat-873',
                'run_uuid': '6e2491bbc96340b7b0061be7354fadea',
                'start_time': 1716630311543,
                'status': 'FINISHED',
                'user_id': 'codespace'},
    'data': {   'metrics': {'accuracy': 0.96},
                'params': {'C': '0.1', 'random_state': '42'},
                'tags': {   'mlflow.user': 'codespace',
                            'mlflow.source.name': '/workspaces/mlops-zoomcamp/.micromamba/envs/mlops/lib/python3.11/site-packages/ipykernel_launcher.py',
                            'mlflow.source.type': 'LOCAL',
                            'mlflow.

In [61]:
pd.DataFrame(client.search_runs(experiment_ids='1')[0].info)

,0,1
0,artifact_uri,/workspaces/mlops-zoomcamp/02-experiment-track...
1,end_time,1716630314138
2,experiment_id,1
3,lifecycle_stage,active
4,run_id,6e2491bbc96340b7b0061be7354fadea
5,run_name,able-shoat-873
6,run_uuid,6e2491bbc96340b7b0061be7354fadea
7,start_time,1716630311543
8,status,FINISHED
9,user_id,codespace


In [50]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Registered model 'iris-classifier' already exists. Creating a new version of this model...
2024/05/25 10:22:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1716632575102, current_stage='None', description='', last_updated_timestamp=1716632575102, name='iris-classifier', run_id='6e2491bbc96340b7b0061be7354fadea', run_link='', source='/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts/1/6e2491bbc96340b7b0061be7354fadea/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>